In [1]:
# Import necessary libraries
import pandas as pd
import yaml
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report


In [2]:
# Go to project root folder
%cd ..

/Users/porfirio.hernandez/Documents/mlop-coecytjal/module 8


/Users/porfirio.hernandez/Documents/mlop-coecytjal/module 8/.mlops/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# Read config
import pprint

with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)

pprint.pprint(config)

{'base': {'log_level': 'INFO', 'random_state': 42},
 'data_load': {'dataset_csv': 'data/raw/penguins.csv',
               'dataset_prepare': 'data/processed/prepare_penguins.csv'},
 'data_split': {'test_size': 0.2,
                'testset_path': 'data/processed/test_penguins.csv',
                'trainset_path': 'data/processed/train_penguins.csv'},
 'evaluate': {'confusion_matrix_image': 'confusion_matrix.png',
              'metrics_file': 'metrics.json',
              'reports_dir': 'reports'},
 'featurize': {'categorical_features': ['sex'],
               'features_path': 'data/processed/featured_penguins.csv',
               'target_column': 'species'},
 'train': {'cv': 3,
           'estimator_name': 'rdm_forest',
           'estimators': {'logreg': {'param_grid': {'C': [0.005],
                                                    'max_iter': [100],
                                                    'multi_class': ['multinomial'],
                                               

### Data Prepare

In [4]:
# Load the Penguins dataset
df = pd.read_csv(config['data_load']['dataset_csv'])

# Display the first few rows of the dataset
df.head()
print(df.shape)
# drop NaNs
df = df.dropna(axis=0, how='any')
df = df.drop('Unnamed: 0', axis=1)


(344, 9)


### Feature Processing

In [19]:
# Split the data into features (X) and target variable (y) and dropping specific columns from the dataset
X = df.drop(['species', 'island', 'year', 'bill_length_mm'], axis=1)
y = df['species']

# Define categorical features
categorical_features = ['sex']

# Create a column transformer with one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

X_processed = preprocessor.fit_transform(X)


In [20]:
X_processed_df = pd.DataFrame(X_processed, columns=preprocessor.get_feature_names_out())

### Data Split

In [23]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

### Model Training

In [24]:
# Create a logistic regression model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
model.fit(X_train, y_train)
joblib.dump(model, config['train']['model_path'])


['models/model.joblib']

### Model Evaluate

In [25]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Display the results
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_rep)


Accuracy: 0.79

Classification Report:
               precision    recall  f1-score   support

      Adelie       0.72      0.90      0.80        31
   Chinstrap       0.70      0.39      0.50        18
      Gentoo       1.00      1.00      1.00        18

    accuracy                           0.79        67
   macro avg       0.81      0.76      0.77        67
weighted avg       0.79      0.79      0.77        67

